使用input_data下载导入MNIST手写数据集。
单层网络softmax回归训练

In [23]:
import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, {x: batch_xs, y_: batch_ys})
    print 'test accuracy: ', sess.run(accuracy, {x: mnist.test.images, y_: mnist.test.labels})


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
test accuracy:  0.9125


添加一个隐藏层

In [26]:
import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

hidden_neurons = 15
x = tf.placeholder(tf.float32, [None, 784])
W1 = tf.Variable(tf.random_normal([784, hidden_neurons], stddev=0.1))
b1 = tf.Variable(tf.random_normal([hidden_neurons], stddev=0.1))
W2 = tf.Variable(tf.random_normal([hidden_neurons, 10], stddev=0.1))
b2 = tf.Variable(tf.random_normal([10], stddev=0.1))
hidden_layer = tf.sigmoid(tf.matmul(x, W1) + b1)
y = tf.nn.softmax(tf.matmul(hidden_layer, W2) + b2)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, {x: batch_xs, y_: batch_ys})
    print 'test accuracy: ', sess.run(accuracy, {x: mnist.test.images, y_: mnist.test.labels})


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
test accuracy:  0.9262


使用CNN

In [ ]:
import input_data
import tensorflow as tf

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W):
    return tf.nn.conv2d(x, W, [1, 1, 1, 1], 'SAME')

def max_pool(x):
    return tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')


mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool(h_conv2)

h_flat = tf.reshape(h_pool2, [-1, 7*7*64])
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(20000):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if i % 100 == 0:
            ac = accuracy.eval({x: batch_xs, y_: batch_ys})
            print "step %d, training accuracy: %g" % (i, ac)
        sess.run(train_step, {x: batch_xs, y_: batch_ys})
    print 'test accuracy: ', sess.run(accuracy, {x: mnist.test.images, y_: mnist.test.labels})


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy: 0.1
step 100, training accuracy: 0.8
step 200, training accuracy: 0.96
step 300, training accuracy: 0.86
step 400, training accuracy: 0.94
step 500, training accuracy: 0.94
step 600, training accuracy: 0.96
step 700, training accuracy: 0.96
step 800, training accuracy: 0.9
step 900, training accuracy: 1
step 1000, training accuracy: 0.96
step 1100, training accuracy: 0.9
step 1200, training accuracy: 0.96
step 1300, training accuracy: 1
step 1400, training accuracy: 0.92
step 1500, training accuracy: 1
step 1600, training accuracy: 0.94
step 1700, training accuracy: 1
